## 1. Import Required Libraries

In [ ]:
from notebookutils import mssparkutils
from pyspark.sql import SparkSession
import logging

## 2. Access Environment Variables

Use `mssparkutils.env.getVariable()` to retrieve values from the active Variable Library.

In [ ]:
# Get environment-specific variables
storage_account = mssparkutils.env.getVariable("storage_account")
api_endpoint = mssparkutils.env.getVariable("api_endpoint")
data_lake_path = mssparkutils.env.getVariable("data_lake_path")
log_level = mssparkutils.env.getVariable("log_level")

# Integer variables
batch_size = int(mssparkutils.env.getVariable("batch_size"))
max_retries = int(mssparkutils.env.getVariable("max_retries"))
timeout_seconds = int(mssparkutils.env.getVariable("timeout_seconds"))

# Boolean variables
enable_caching = mssparkutils.env.getVariable("enable_caching").lower() == "true"

print("Environment Variables Loaded:")
print(f"  Storage Account: {storage_account}")
print(f"  API Endpoint: {api_endpoint}")
print(f"  Data Lake Path: {data_lake_path}")
print(f"  Log Level: {log_level}")
print(f"  Batch Size: {batch_size}")
print(f"  Max Retries: {max_retries}")
print(f"  Timeout: {timeout_seconds}s")
print(f"  Caching Enabled: {enable_caching}")

## 3. Configure Logging with Variable

Use the `log_level` variable to configure logging dynamically per environment.

In [ ]:
# Configure logging based on environment
log_level_map = {
    "DEBUG": logging.DEBUG,
    "INFO": logging.INFO,
    "WARNING": logging.WARNING,
    "ERROR": logging.ERROR
}

logging.basicConfig(
    level=log_level_map.get(log_level, logging.INFO),
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)
logger.info(f"Logging configured at {log_level} level")
logger.debug("Debug messages visible in DEV only")

## 4. Use Variables in Data Processing

Apply batch size and storage path variables in Spark operations.

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("VariableLibraryExample").getOrCreate()

# Configure Spark with caching based on variable
if enable_caching:
    spark.conf.set("spark.sql.adaptive.enabled", "true")
    logger.info("Spark caching enabled")

# Create sample data
data = [(i, f"name_{i}", i * 10) for i in range(1, 10001)]
columns = ["id", "name", "value"]
df = spark.createDataFrame(data, columns)

# Apply batch size from variable
df_limited = df.limit(batch_size)

logger.info(f"Processing {df_limited.count()} rows (batch_size={batch_size})")
df_limited.show(10)

## 5. Read from Data Lake Using Variable Path

In [ ]:
# Construct path using variable
# Note: In real scenario, this would read from actual data lake
# This example shows the pattern

try:
    # Example: Read parquet from data lake
    # df_lake = spark.read.parquet(f"{data_lake_path}raw/sales/")
    
    logger.info(f"Would read from: {data_lake_path}raw/sales/")
    logger.info(f"Storage account: {storage_account}")
    
except Exception as e:
    logger.warning(f"Data lake read example (not connected): {e}")

## 6. API Integration with Variable Endpoint

In [ ]:
import requests
from time import sleep

def call_api_with_retry(endpoint: str, max_retries: int, timeout: int):
    """
    Call API with retry logic using variables from Variable Library
    """
    for attempt in range(1, max_retries + 1):
        try:
            logger.info(f"API call attempt {attempt}/{max_retries}")
            
            # Example API call (would need actual endpoint)
            # response = requests.get(endpoint, timeout=timeout)
            # response.raise_for_status()
            
            logger.info(f"Would call: {endpoint} (timeout={timeout}s)")
            return True
            
        except Exception as e:
            logger.warning(f"Attempt {attempt} failed: {e}")
            if attempt < max_retries:
                sleep(2 ** attempt)  # Exponential backoff
    
    logger.error(f"All {max_retries} attempts failed")
    return False

# Use variables in API call
success = call_api_with_retry(
    endpoint=api_endpoint,
    max_retries=max_retries,
    timeout=timeout_seconds
)

print(f"API integration configured: {success}")

## 7. Dynamic Configuration Summary

Show how variables enable environment-specific behavior without code changes.

In [ ]:
print("\n" + "="*60)
print("ENVIRONMENT CONFIGURATION SUMMARY")
print("="*60)

print(f"\nStorage Configuration:")
print(f"  Account: {storage_account}")
print(f"  Path: {data_lake_path}")

print(f"\nAPI Configuration:")
print(f"  Endpoint: {api_endpoint}")
print(f"  Timeout: {timeout_seconds}s")
print(f"  Max Retries: {max_retries}")

print(f"\nProcessing Configuration:")
print(f"  Batch Size: {batch_size} rows")
print(f"  Caching: {'Enabled' if enable_caching else 'Disabled'}")
print(f"  Log Level: {log_level}")

print(f"\nEnvironment Detection:")
if 'dev' in storage_account.lower():
    print("  Running in: DEVELOPMENT")
    print("  Characteristics: Debug logging, small batches, test endpoints")
elif 'uat' in storage_account.lower():
    print("  Running in: UAT")
    print("  Characteristics: Info logging, medium batches, UAT endpoints")
elif 'prod' in storage_account.lower():
    print("  Running in: PRODUCTION")
    print("  Characteristics: Warning logging, large batches, prod endpoints")
else:
    print("  Running in: UNKNOWN")

print("\n" + "="*60)
print("✓ All variables loaded successfully from Variable Library")
print("✓ Configuration applied dynamically based on environment")
print("✓ No code changes needed between environments")
print("="*60)

## Key Benefits

1. **Runtime Flexibility**: Change variables without redeploying notebooks
2. **Environment Parity**: Same code runs in dev/uat/prod with different variables
3. **Centralized Management**: Update variables in Fabric portal for all artifacts
4. **Type Safety**: Explicit type conversion for Int and Bool variables
5. **Separation of Concerns**: Code contains logic, Variable Library contains configuration

## Next Steps

- Update your existing notebooks to use Variable Library variables
- Replace hard-coded values with `mssparkutils.env.getVariable()` calls
- Test in development environment
- Promote to UAT/Production with environment-specific Variable Libraries